In [ ]:
import sqlite3
import pandas as pd
import datetime
# Create your connection.
cnx = sqlite3.connect('/content/drive/MyDrive/database.sqlite')
#df_p = pd.read_sql_query("SELECT * FROM podcasts",cnx)
#df_r = pd.read_sql_query("SELECT * FROM reviews",cnx) 
df = pd.read_sql_query("SELECT pod.podcast_id, pod.title,rev.author_id, cat.category, pod.slug, pod.itunes_id, rev.rating, rev.content, rev.title as Review_Title, rev.created_at FROM reviews rev join podcasts pod on rev.podcast_id = pod.podcast_id join categories cat on pod.podcast_id = cat.podcast_id limit 27000", cnx)
#df = pd.read_sql_query("SELECT  * FROM reviews rev join podcasts pod on rev.podcast_id = pod.podcast_id join categories cat on pod.podcast_id = cat.podcast_id limit 30000", cnx)
#df.to_csv("/content/drive/MyDrive/Podcast_10.csv", index = True,)

In [ ]:
df.head(3)

,podcast_id,title,author_id,category,slug,itunes_id,rating,content,Review_Title,created_at
0,c61aa81c9b929a66f0c1db6cbe5d8548,Backstage at Tilles Center,F7E5A318989779D,arts,backstage-at-tilles-center,1373261997,5,Thanks for providing these insights. Really e...,really interesting!,2018-04-24T12:05:16-07:00
1,c61aa81c9b929a66f0c1db6cbe5d8548,Backstage at Tilles Center,F7E5A318989779D,arts-performing-arts,backstage-at-tilles-center,1373261997,5,Thanks for providing these insights. Really e...,really interesting!,2018-04-24T12:05:16-07:00
2,c61aa81c9b929a66f0c1db6cbe5d8548,Backstage at Tilles Center,F7E5A318989779D,music,backstage-at-tilles-center,1373261997,5,Thanks for providing these insights. Really e...,really interesting!,2018-04-24T12:05:16-07:00


In [ ]:
df.shape

(27000, 10)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer(stop_words='english') 
#stop_words in english for Ignoring 'The' ,'a' etc. 
df['new'] = df['title']+' '+df['category']+' '+df['Review_Title']
df['new'] = df['new'].fillna('')
tfidf_matrix = tfidf.fit_transform(df['new'])


In [ ]:
print(tfidf_matrix)

  (0, 1999)	0.2954715132528008
  (0, 3013)	0.3247199516107557
  (0, 280)	0.0910286864093135
  (0, 597)	0.5275551161516907
  (0, 3721)	0.5275551161516907
  (0, 340)	0.4922788995853425
  (1, 2741)	0.1822213340407624
  (1, 1999)	0.28697952477947164
  (1, 3013)	0.31538734943946195
  (1, 280)	0.1768249593976499
  (1, 597)	0.5123929371785422
  (1, 3721)	0.5123929371785422
  (1, 340)	0.4781305754544636
  (2, 2504)	0.2855628387840395
  (2, 1999)	0.2843486141385568
  (2, 3013)	0.31249600750735773
  (2, 597)	0.5076955287770594
  (2, 3721)	0.5076955287770594
  (2, 340)	0.47374727034000796
  (3, 1998)	0.44024384713196046
  (3, 2235)	0.2626775096539272
  (3, 280)	0.1713585703555125
  (3, 597)	0.4965527574516377
  (3, 3721)	0.4965527574516377
  (3, 340)	0.46334958668863296
  :	:
  (26995, 280)	0.11719525611741381
  (26996, 3444)	0.6446005963000198
  (26996, 2767)	0.2859669012980881
  (26996, 3954)	0.3043864992430448
  (26996, 3999)	0.43663291806286514
  (26996, 1660)	0.21867807046537555
  (26996, 28

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
#Since these two matrices are similar to each other, we wrote both of them at the same time.
cosine_sim.shape


(27000, 27000)

In [ ]:
print(cosine_sim[1])

array([0.97930761, 1.        , 0.92695178, ..., 0.        , 0.03657275,
       0.06054637])

In [ ]:
df = df[~df["title"].isna()]
#Deletion of titles with Nan Values
indices = pd.Series(df.index, index=df['title'])
#Let's note the titles of all the podcast.
indices.head()

title
Backstage at Tilles Center    0
Backstage at Tilles Center    1
Backstage at Tilles Center    2
Backstage at Tilles Center    3
Backstage at Tilles Center    4
dtype: int64

In [ ]:
indices.shape
#Delete duplicated ones.
indices = indices[~indices.index.duplicated(keep='last')]
indices.shape

indices[:10]

title
Backstage at Tilles Center                                   5
TED Talks Daily                                            537
Snap Judgment                                             1956
Raging Bullets                                            2284
Selected Shorts                                           2911
Cameralabs photography podcast                            2935
BACKSTORY                                                 2992
Comic Geek Speak Podcast - The Best Comic Book Podcast    3092
Homecoming                                                3239
The Comic Conspiracy                                      3362
dtype: int64

In [ ]:
pod_index = indices["Cameralabs photography podcast"]
cosine_sim[pod_index]

array([0.        , 0.        , 0.        , ..., 0.18134852, 0.18725225,
       0.15499851])

In [ ]:
similarity_scores = pd.DataFrame(cosine_sim[pod_index], columns=["similarity_score"])
similarity_scores.sort_values(by='similarity_score',ascending=False).head(10)

,similarity_score
2935,1.000000
2933,0.961574
2934,0.934293
2929,0.499235
2927,0.417105
26616,0.393916
26709,0.393916
26973,0.393916
26844,0.393916
2928,0.365710


In [ ]:
pod_indices = similarity_scores.sort_values("score", ascending=False)[1:300].index
df_new = df['title'][pod_indices]
pd.DataFrame(df_new.unique(), columns = ['title'])

,title
0,Cameralabs photography podcast
1,B&H Photography Podcast
2,The Forward
3,PHOTOGRAPHY TIPS FROM THE TOP FLOOR
4,The Sporkful
5,I Should Be Writing
6,Geek's Guide to the Galaxy - A Science Fiction...
7,House of Carbs
